# Analysis summary & result

- 원본 데이터 요약
  - 특정시점(2017-10-30) challenge, master tier 계정이 7.21버전으로 플레이한 게임을 모았다. (5985 게임)
  - 그 중에서 소환사의 협곡, 솔로랭크, Classic game mode만으로 대상을 추렸다. (3051 게임)
  - 팀별 픽한 챔피언과 승리 여부를 모았다.
- 분석방향
  - Bayes’ theorem을 활용하여 주어진 상황에서 내가 어떤 챔피언을 고를 경우 승률을 예측하고 싶다.
  - Expect win rate(= posterior probability)
    - Champion win rate(= prior probability) * Pick champion win rate(= likelihood) / Champions win rate(= evidence)
    - 우리팀과 상대팀이 챔피언 픽을 한 상태에서 내 픽 차례가 왔을 경우 특정 챔피언의 승률
      - Champion win rate(= prior probability)
        - 한 챔피언을 골랐을 때 승률
      - Champions win rate(= evidence)
        - 우리팀과 상대팀이 챔피언들을 골랐을 때 승률
      - Pick champion win rate(= likelihood)
        - 내가 고른 챔피언이 있는 경우에 한해 내가 픽하기 전에 우리팀과 다른팀 픽한 챔피언들이 있는 게임의 승률
  - Recommend champion
    - Expect win rate(= posterior probability)을 이용해 어떤 챔피언을 내가 픽 했을 때 가장 승률 계산
    - 내림차순으로 상위 5개 챔피언을 보여준다.
    - 다만 게임 플레이 수가 300회 이상인 챔피언만 보여준다.
      - 아트록스 같은 경우 일반적으로 좋지 않은 챔피언이지만 장인들이 사용해서 게임수는 적으나 승률이 높을 수 있다.
    - 벤이 되어 있지 않은 챔피언을 선택하고 플레이한다.
- 결론
  - Bayesian view 픽 추천 시스템을 만들었다.
    - 최초에는 벤 추천 시스템까지 만들려 했었지만 프로리그에서의 벤픽과 달리 랭크 게임에서는 각 플레이어가 동시에 한 케릭터씩 밴을 하기 때문에 분석의 의미가 없다고 판단했다.
      - 벤픽 순서가 있는 프로리그 데이터가 있다면 프로리그용 벤픽 추천 시스템을 만들 수 있다.
  - Bayesian view는 데이터가 많이 쌓이면 쌓일 수록 강력해지기에 셈플 데이터의 숫자가 중요하다.
    - 셈플 숫자가 부족하여 (0,1), (1,2) 상황에서의 픽 이후 단계는 잘 추천해주지 못한다.
- 부족한점
  - 챔피언을 고르는 행위가 어떤 챔피언이 앞에 픽이 되었는지와 상관없이 독립적인 선택이라는 강력한 가정이 들어가 있다.
    - 하지만 현실은 우리팀이 알리스타나 말파이트를 고르면 야스오 픽이지.
  - 본인의 포지션별로 추천해주지는 못한다. 이것 역시도 셈플이 더 많다고 하면 포지션에 따른 픽 셈플을 가지고 분석이 가능하다.
  - 프로게이머들에게는 부적합하다. 개인화되어야 할 필요가 있다.
  - 함수가 세부 기능 단위로 나뉘어져 있지 않아서 가독성이 떨어진다.
  - 현재 Riot에서 제공하는 데이터로는 픽 순서를 파악할 수 없다.
- 느낀점
  - 난 피지컬이 부족하기에 어떻게 하면 피지컬을 제외한 부분에서 효과적인 전략을 세울 수 있는 부분이 무엇일까를 고민하다 본 분석을 시작하게 되었다.
    - 하지만 그 챔피언의 숙련도가 떨어진다면 소용없다는 건 함정.
  - 프로게임단 코치들은 벤픽 전략을 어떻게 세우는지 궁금하다.
  - 2017.11.04 2017년 롤드컵 결승전이 얼마 안남았다. SKT 화이팅! 울프 선수 화이팅!

In [1]:
import json
import requests
import numpy as np
import pandas as pd
import time
import datetime

In [2]:
pd.set_option('display.max_rows', 20)

In [3]:
with open('analysis_dataset.json', 'r') as js:
     games = json.load(js)
len(games)

3051

In [4]:
gameId = '2994319360'
games[gameId]

{'ban': {'100': [8, 429, 421, 497, 60], '200': [114, 7, 38, 429, 31]},
 'pick': {'100': [18, 245, 48, 69, 150], '200': [202, 103, 113, 12, 157]},
 'pick_role': {'100': {'BOTTOM_DUO_CARRY': 18,
   'BOTTOM_DUO_SUPPORT': 48,
   'JUNGLE_NONE': 245,
   'MIDDLE_SOLO': 69,
   'TOP_SOLO': 150},
  '200': {'BOTTOM_DUO_CARRY': 202,
   'BOTTOM_DUO_SUPPORT': 12,
   'JUNGLE_NONE': 113,
   'MIDDLE_SOLO': 103,
   'TOP_SOLO': 157}},
 'win': {'100': 'Fail', '200': 'Win'}}

In [5]:
df_champion = pd.read_csv('champion_info.csv')
df_champion.column = ['champion','championId']
df_champion.tail()

,key,id
133,Shen,98
134,JarvanIV,59
135,Malzahar,90
136,Zac,154
137,Gragas,79


In [6]:
champion_id_list = list(df_champion.id)
champion_id_list.sort()

# Champion win rate(= prior probability)

In [7]:
def safe_divide(x, y):
    """
    prevent from dividing by 0
    :param x: numerator
    :param y: denominator
    :return: float
    """
    try:
        return round(x / y, 4)
    except ZeroDivisionError:
        return 0

In [8]:
def champion_win_rate(champion_id):
    """
    챔피언 하나의 승률.
    :param champion_id: championId
    :return: [승률, 챔피언이 플레이한 게임수, 챔피언이 플레이해서 이긴 게임수]
    """
    game_count = 0
    win_count = 0
    teamIds = ['100', '200']
    for gameId in games.keys():
        for teamId in teamIds:
            if champion_id in games[gameId]['pick'][teamId]:
                game_count += 1
                if games[gameId]['win'][teamId] == 'Win':
                    win_count += 1
    return [safe_divide(win_count, game_count), game_count, win_count]

In [9]:
champion_win_rate(498)

[0.5211, 1019, 531]

In [10]:
champion_win_rates = pd.DataFrame()
# champion_win_rates = dict()
for champion_id in champion_id_list:
    champion_win_rates = champion_win_rates.append([[champion_id, 
                                                     champion_win_rate(champion_id)[0],
                                                     champion_win_rate(champion_id)[1],
                                                     champion_win_rate(champion_id)[2]]])
champion_win_rates.columns = ['champion_id', 'champion_win_rate', 'game_count', 'champion_win_count']
champion_win_rates.reset_index(inplace=True)
del champion_win_rates['index']

In [11]:
champion_win_rates.sort_values('champion_win_rate', ascending=False)

,champion_id,champion_win_rate,game_count,champion_win_count
63,72,1.0000,1,1
126,266,0.6875,16,11
51,56,0.6667,6,4
29,30,0.6258,155,97
31,32,0.6250,8,5
46,50,0.5833,36,21
88,105,0.5746,134,77
10,11,0.5641,39,22
3,4,0.5605,430,241
23,24,0.5603,489,274


# Champions win rate(= evidence)

In [12]:
def champions_win_rate(my_team_champions, opponent_team_champions):
    """
    우리 팀과 적 팀이 고른 챔피언이 속한 게임의 승률.
    항상 우리 팀이 고른 챔피언 숫자가 적을 경우 픽하는 상황으로 한정한다.
    실 게임에서는 동시에 2명이 픽 하는 경우, 다른 summoner가 먼저 픽 할 경우 고른 챔피언 숫자가 우리팀과 적팀이 같으나 여기선 무시한다.
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: [승률, 플레이한 게임수, 플레이해서 이긴 게임수]
    """
    if (len(my_team_champions), len(opponent_team_champions)) in [(0,1),(1,2),(2,3),(3,4),(4,5)]:
        if (len(my_team_champions), len(opponent_team_champions)) in [(0,1)]:
            return champion_win_rate(opponent_team_champions[0])
        else:
            game_count = 0
            win_count = 0
            teamIds = ['100', '200']
            for gameId in games.keys():
                for teamId in teamIds:
                    # 우리 팀이 뽑은 챔피언이 games안에 모두 있는지 확인한다.
                    if len(set(games[gameId]['pick'][teamId]) - set(my_team_champions)) == (5 - len(my_team_champions)):
                        # 적 팀이 뽑은 챔피언이 반대 팀에 있는지 확인한다.
                        if len(set(games[gameId]['pick'][list(set(teamIds) - set([teamId]))[0]]) - 
                               set(opponent_team_champions)) == (5 - len(opponent_team_champions)):
                            # 우리 팀과 적 팀이 뽑은 챔피언이 모두 있는 경우는 경기수 증가
                            game_count += 1
                            if games[gameId]['win'][teamId] == 'Win':
                                # 우리 팀이 이긴 경우 승수 추가
                                win_count += 1
            return [safe_divide(win_count, game_count), game_count, win_count]
    else:
        return print('''Input correct value. Only (len(my_team_champions), len(opponent_team_champions)) 
in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.''')

In [13]:
champions_win_rate([], [59])

[0.4864, 1398, 680]

In [14]:
champions_win_rate([24], [59, 110])

[0.9231, 13, 12]

In [15]:
champions_win_rate([24, 56], [59, 110])

Input correct value. Only (len(my_team_champions), len(opponent_team_champions)) 
in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.


# Pick champion win rate(= likelihood)

In [16]:
def pick_champion_win_rate(my_champion ,my_team_champions, opponent_team_champions):
    """
    내가 고른 챔피언이 있는 게임들 중에서 우리 팀과 적 팀이 고른 챔피언이 속한 게임의 승률.
    항상 우리 팀이 고른 챔피언 숫자가 적을 경우 픽하는 상황으로 한정한다.
    실 게임에서는 동시에 2명이 픽 하는 경우, 다른 summoner가 먼저 픽 할 경우 고른 챔피언 숫자가 우리팀과 적팀이 같으나 여기선 무시한다.
    :param my_champion: 내가 고른 championId int
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: [승률, 플레이한 게임수, 플레이해서 이긴 게임수]
    """
    # 같은팀 두명이 동시에 고르는 경우 내가 픽 했더라도 팀원은 아직 고르지 않았다는 가정을 한다.
    if my_champion in my_team_champions:
        print('Pick another champion. The champion is already picked by teammate.')
        return [0, 0, 0]
    else:
        if my_champion in opponent_team_champions:
            print('Pick another champion. The champion is already picked by opponent.')
            return [0, 0, 0]
        else:
            if (len(my_team_champions) + 1, len(opponent_team_champions)) in [(1,1),(2,2),(3,3),(4,4),(5,5)]:
                new_my_team_champions = my_team_champions + [my_champion]
                game_count = 0
                win_count = 0
                teamIds = ['100', '200']
                for gameId in games.keys():
                    for teamId in teamIds:
                        # 내가 새로 뽑을 챔피언과 우리 팀이 뽑은 챔피언이 games안에 모두 있는지 확인한다.
                        if len(set(games[gameId]['pick'][teamId]) - set(new_my_team_champions)) == \
                            (5 - len(new_my_team_champions)):
                            # 적 팀이 뽑은 챔피언이 반대 팀에 있는지 확인한다.
                            if len(set(games[gameId]['pick'][list(set(teamIds) - set([teamId]))[0]]) - 
                                   set(opponent_team_champions)) == (5 - len(opponent_team_champions)):
                                # 우리 팀과 적 팀이 뽑은 챔피언이 모두 있는 경우는 경기수 증가
                                game_count += 1
                                if games[gameId]['win'][teamId] == 'Win':
                                    # 우리 팀이 이긴 경우 승수 추가
                                    win_count += 1
                return [safe_divide(win_count, game_count), game_count, win_count]
            else:
                return print('''Input correct value. Only (len(my_team_champions), len(opponent_team_champions)) 
in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.''')

In [17]:
pick_champion_win_rate(498, [], [59])

[0.5161, 248, 128]

In [18]:
pick_champion_win_rate(498, [240], [59,110])

[0, 0, 0]

In [19]:
pick_champion_win_rate(96, [113, 90, 63], [59, 110, 80, 134])

[1.0, 1, 1]

In [20]:
pick_champion_win_rate(90, [113, 90, 63], [59, 110, 80, 134])

Pick another champion. The champion is already picked by teammate.


[0, 0, 0]

In [21]:
pick_champion_win_rate(59, [113, 90, 63], [59, 110, 80, 134])

Pick another champion. The champion is already picked by opponent.


[0, 0, 0]

In [22]:
pick_champion_win_rate(59, [113, 90, 63, 100], [110, 80, 134])

Input correct value. Only (len(my_team_champions), len(opponent_team_champions)) 
in [(0,1), (1,2), (2,3), (3,4), (4,5)] is allowed.


# Expect win rate(= posterior probability)

In [23]:
def expect_win_rate(my_champion, my_team_champions, opponent_team_champions):
    """
    내가 특정 챔피언을 고를 경우 우리 팀과 적 팀이 선택한 챔피언들을 바탕으로 예상되는 승률.
    :param my_champion: 내가 고른 championId int
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: 예상 승률
    """
    upper = (pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[0] * 
             champion_win_rate(my_champion)[0])
    lower = champions_win_rate(my_team_champions, opponent_team_champions)[0]
    if np.logical_or(lower==0, pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions)[1]==0):
        return print('Can not calculate. Lack of samples')
    else:
        return safe_divide(upper, lower)

In [24]:
my_champion = 59
my_team_champions = [18]
opponent_team_champions = [498, 81]
print('prior probability:',champion_win_rate(my_champion))
print('likelihood:', pick_champion_win_rate(my_champion, my_team_champions, opponent_team_champions))
print('evidence:', champions_win_rate(my_team_champions, opponent_team_champions))
print('posterior probability:', expect_win_rate(my_champion, my_team_champions, opponent_team_champions))

prior probability: [0.4864, 1398, 680]
likelihood: [0.7778, 9, 7]
evidence: [0.5556, 36, 20]
posterior probability: 0.6809


# Recommend champion

In [25]:
df_champions = pd.read_csv('champion_info.csv')
df_champions.tail()

,key,id
133,Shen,98
134,JarvanIV,59
135,Malzahar,90
136,Zac,154
137,Gragas,79


In [26]:
def championName_to_championId(championName):
    """
    챔피언 이름을 영문으로 넣으면 championId가 나온다.
    :param championName: 챔피언 이름 string
    :return: championId int
    """
    return df_champions.set_index('key').to_dict()['id'][championName]

In [27]:
championName_to_championId('Ahri')

103

In [28]:
def championNamelist_to_championIdlist(championNamelist):
    """
    영문으로 된 챔피언 이름 리스트를 넣으면 championId의 리스트가 나온다.
    :param championNamelist: 챔피언 이름 list
    :return: championId list
    """
    return list(pd.Series(championNamelist).apply(lambda x: championName_to_championId(x)))

In [29]:
championNamelist_to_championIdlist(['Ahri', 'Anivia'])

[103, 34]

In [30]:
def recommend_championId(my_team_champions, opponent_team_champions):
    """
    내가 픽 할 상황에서 우리편과 상대편의 픽 된 챔피언 리스트를 넣으면 승률이 가장 높을 것으로 예상되는 상위 5개 챔피언을 보여준다.
    :param my_team_champions: 우리 팀의 championId list
    :param opponent_team_champions: 적 팀의 championId list
    :return: DataFrame
    """
    df = pd.DataFrame()
    for champion_Id in champion_win_rates[champion_win_rates.game_count >= 300].champion_id:
        df = df.append([[champion_Id, expect_win_rate(champion_Id, my_team_champions, opponent_team_champions)]])
    df.columns = ['id', 'win_rate']
    df.reset_index(inplace=True)
    del df['index']
    df.sort_values('win_rate', ascending=False, inplace=True)
    df_champions = pd.read_csv('champion_info.csv')
    return pd.merge(df, df_champions, on ='id', how='inner').head()

In [31]:
recommend_championId([113], [59, 81])

Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Pick another champion. The champion is already picked by opponent.
Pick another champion. The champion is already picked by opponent.
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Pick another champion. The champion is already picked by opponent.
Pick another champion. The champion is already picked by opponent.
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Pick another champion. The champion is already picked by teammate.
Pick another champion. The champion is already picked by teammate.
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can

,id,win_rate,key
0,12,0.8882,Alistar
1,134,0.8734,Syndra
2,202,0.8160,Jhin
3,268,0.7867,Azir
4,89,0.7723,Leona


In [32]:
def recommend_championName(my_team_champions, opponent_team_champions):
    '''
    내가 픽 할 상황에서 우리편과 상대편의 픽 된 챔피언 리스트를 넣으면 승률이 가장 높을 것으로 예상되는 상위 5개 챔피언을 보여준다.
    :param my_team_champions: 우리 팀의 championName list
    :param opponent_team_champions: 적 팀의 championName list
    :return: DataFrame
    '''
    my_team_champions = championNamelist_to_championIdlist(my_team_champions)
    opponent_team_champions = championNamelist_to_championIdlist(opponent_team_champions)
    df = pd.DataFrame()
    for champion_Id in champion_win_rates[champion_win_rates.game_count >= 300].champion_id:
        df = df.append([[champion_Id, expect_win_rate(champion_Id, my_team_champions, opponent_team_champions)]])
    df.columns = ['id', 'win_rate']
    df.reset_index(inplace=True)
    del df['index']
    df.sort_values('win_rate', ascending=False, inplace=True)
    df_champions = pd.read_csv('champion_info.csv')
    return pd.merge(df, df_champions, on ='id', how='inner').head()

In [33]:
recommend_championName(['Alistar'], ['Jhin', 'Leona'])

Can not calculate. Lack of samples
Pick another champion. The champion is already picked by teammate.
Pick another champion. The champion is already picked by teammate.
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Pick another champion. The champion is already picked by opponent.
Pick another champion. The champion is already picked by opponent.
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Can not calculate. Lack of samples
Pick another champion. The champ

,id,win_rate,key
0,24,0.7004,Jax
1,76,0.6811,Nidalee
2,8,0.6659,Vladimir
3,81,0.6550,Ezreal
4,29,0.6354,Twitch
